# Database — Report

This notebook visualizes the CSV outputs generated by the **Database** block.

## What this notebook shows
- **Tables by mapped entities**: Treemap and Bar (same metric in two formats).
- **Field annotations**: Global top annotations (Bar).
- **Relationship statistics**: Top entities by relationship count (Bar) and distribution (Histogram).
- **Entity → Entity**: Optional Sankey if edges data is available.

> If a CSV is missing or empty, the cell prints an info message and skips the chart.


In [ ]:
# Setup: imports, paths, helpers
# - CSVs are read from reports/csv-reports/<CATEGORY>/<file>.csv relative to this notebook folder.
# - Minimal console output; only show information if a CSV is missing/empty.
# - Bar charts use an explicit default color so it's easy to tweak later.
# - Titles are standardized without block prefixes.

import os, ast
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('future.no_silent_downcasting', True)

CATEGORY = "Database"
CSV_BASE = Path("../reports/csv-reports").resolve()
DB_DIR = CSV_BASE / CATEGORY

# Explicit default color for all bar charts in this notebook
DEFAULT_BAR_COLOR = ["#1f77b4"]

# CSV IO helpers
NA_LITS = ["", " ", "NA", "N/A", "n/a", "NaN", "NULL", "Null", "null", "None", "none", "-", "--"]

def read_csv_safe(p: Path) -> pd.DataFrame:
    """Read a CSV if present; otherwise return an empty DataFrame.
    Prints a minimal info message when missing or unreadable."""
    p = Path(p)
    if not p.exists():
        print(f"[info] Missing CSV: {p}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(p, na_values=NA_LITS, keep_default_na=True)
        df.columns = [str(c).strip() for c in df.columns]
        df = df.dropna(how="all")
        return df
    except Exception as e:
        print(f"[warn] Failed to read {p}: {e}")
        return pd.DataFrame()

def labelize_na(s, label="N/A"):
    s = s.copy()
    s = s.mask(s.isna(), label).astype(str)
    s = s.replace({"nan": label, "NaN": label})
    return s

def parse_listlike(x):
    """Return a list from cell x tolerant to JSON/Python lists or common separators."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if isinstance(x, (list, tuple, set)):
        return [str(i).strip() for i in x if str(i).strip()]
    s = str(x).strip()
    if not s or s in {"N/A","NA","null","None"}:
        return []
    if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
        try:
            val = ast.literal_eval(s)
            if isinstance(val, (list, tuple, set)):
                return [str(i).strip() for i in val if str(i).strip()]
        except Exception:
            pass
    for sep in [";", ",", "|"]:
        if sep in s:
            return [t.strip() for t in s.split(sep) if t.strip()]
    return [s]

def find_col(df, *cands, default=None, contains=None):
    """Find a column by exact candidates or by substring (contains)."""
    low = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in low:
            return low[c.lower()]
    if contains:
        for k, orig in low.items():
            if contains.lower() in k:
                return orig
    return default

TOP_N = 40
MAX_BARS = 25  # cap for long bar charts


## 1) Tables by Number of Mapped Entities

In [ ]:
# Charts for Jpa_Entities
# Where charts are generated (same metric, two visual encodings):
#  - 1A) Tables by Number of Mapped Entities (Treemap)
#  - 1B) Tables by Number of Mapped Entities (Bar, explicit color)

path = DB_DIR / "Jpa_Entities.csv"
df_ent = read_csv_safe(path)
if not df_ent.empty:
    c_entity = find_col(df_ent, "Entity", contains="entity", default="Entity")
    c_table  = find_col(df_ent, "TableName", contains="table", default=None)

    if c_table is None:
        print("[info] No TableName column found — skipping charts.")
    else:
        tmp = pd.DataFrame({
            "entity":  df_ent[c_entity].astype(str),
            "table":   labelize_na(df_ent[c_table]),
            "value":   1
        })
        agg = tmp.groupby("table")["value"].sum().reset_index(name="entities")

        # 1A) Treemap — same metric as bar, alternative representation
        fig_treemap = px.treemap(agg, path=["table"], values="entities",
                                 title="Tables by Number of Mapped Entities (Treemap)")
        fig_treemap.update_layout(width=1000, height=600)
        fig_treemap.show()

        # 1B) Bar — same metric as treemap, alternative representation
        top_tabs = agg.sort_values("entities", ascending=False).head(MAX_BARS)
        fig_bar = px.bar(top_tabs, x="table", y="entities", text="entities",
                         title="Tables by Number of Mapped Entities (Bar)",
                         color_discrete_sequence=DEFAULT_BAR_COLOR)
        fig_bar.update_traces(textposition="outside", cliponaxis=False)
        fig_bar.update_layout(xaxis_tickangle=-45, width=1100, height=550,
                              xaxis_title="Table", yaxis_title="Entities")
        fig_bar.show()
else:
    print("[info] Jpa_Entities.csv missing or empty.")


## 2) Field Annotations — Global Top

In [ ]:
# Charts for Entity_Fields
# Where chart is generated:
#  - 2A) Top Field Annotations (Bar, explicit color)

path = DB_DIR / "Entity_Fields.csv"
df_fields = read_csv_safe(path)

if not df_fields.empty:
    c_entity = find_col(df_fields, "Entity", contains="entity", default="Entity")
    c_ann    = find_col(df_fields, "Annotations", contains="annotation", default=None)

    if c_ann is None:
        print("[info] No annotation-like column found — skipping chart.")
    else:
        rows = []
        for _, r in df_fields.iterrows():
            ent = str(r.get(c_entity, ""))
            for ann in parse_listlike(r.get(c_ann)):
                rows.append({"entity": ent, "annotation": ann})
        ann_df = pd.DataFrame(rows)

        if ann_df.empty:
            print("[info] No annotation entries parsed — skipping chart.")
        else:
            top_ann = (ann_df.groupby("annotation").size()
                       .reset_index(name="count")
                       .sort_values("count", ascending=False)
                       .head(MAX_BARS))
            fig = px.bar(top_ann, x="annotation", y="count", text="count",
                         title="Top Field Annotations",
                         color_discrete_sequence=DEFAULT_BAR_COLOR)
            fig.update_traces(textposition="outside", cliponaxis=False)
            fig.update_layout(xaxis_tickangle=-40, width=1100, height=550,
                              xaxis_title="Annotation", yaxis_title="Count")
            fig.show()
else:
    print("[info] Entity_Fields.csv missing or empty.")


## 3) Relationship Statistics

In [ ]:
# Charts for DB_Schema
# Where charts are generated:
#  - 3A) Top Entities by Relationships (Bar, explicit color)
#  - 3B) Distribution of Relationships per Entity (Histogram)

path = DB_DIR / "DB_Schema.csv"
df_schema = read_csv_safe(path)

if not df_schema.empty:
    c_entity = find_col(df_schema, "Entity", contains="entity", default="Entity")
    c_rel    = find_col(df_schema, "Relationships", contains="relationship", default=None)

    if c_rel is None:
        print("[info] No 'Relationships' column found — skipping stats.")
    else:
        rel_series = pd.to_numeric(df_schema[c_rel], errors="coerce").fillna(0)
        df_rel = pd.DataFrame({
            "Entity": df_schema[c_entity].astype(str),
            "Relationships": rel_series.astype(int)
        })

        # 3A) Top entities by relationships
        top_rel = df_rel.sort_values("Relationships", ascending=False).head(MAX_BARS)
        fig = px.bar(top_rel, x="Entity", y="Relationships", text="Relationships",
                     title="Top Entities by Relationships",
                     color_discrete_sequence=DEFAULT_BAR_COLOR)
        fig.update_traces(textposition="outside", cliponaxis=False)
        fig.update_layout(xaxis_tickangle=-45, width=1200, height=550,
                          xaxis_title="Entity", yaxis_title="Relationships")
        fig.show()

        # 3B) Distribution (histogram)
        fig2 = px.histogram(df_rel, x="Relationships", nbins=20,
                            title="Distribution of Relationships per Entity")
        fig2.update_layout(width=900, height=450, xaxis_title="Relationships", yaxis_title="Count")
        fig2.show()
else:
    print("[info] DB_Schema.csv missing or empty.")


## 4) Entity → Entity by Relation (Optional)

In [ ]:
# Chart for Entity_Relationship_Edges (optional)
# Where chart is generated:
#  - 4A) Entity → Entity by Relation (Sankey), only if edges CSV has valid rows

path = DB_DIR / "Entity_Relationship_Edges.csv"
df_edges = read_csv_safe(path)

# Treat header-only as empty
if not df_edges.empty and len(df_edges.dropna(how="all")) > 0 and len(df_edges.columns) >= 3:
    cols = {c.lower(): c for c in df_edges.columns}
    c_from = cols.get("fromentity") or "fromEntity"
    c_to   = cols.get("toentity") or "toEntity"
    c_rel  = cols.get("relation") or "relation"

    e2e = df_edges.dropna(subset=[c_from, c_to])
    if e2e.empty:
        print("[info] Edges CSV has only empty rows — skipping Sankey.")
    else:
        g = e2e.groupby([c_from, c_to, c_rel]).size().reset_index(name="count")
        # Cap nodes by degree to keep legible
        deg = pd.concat([g.groupby(c_from)["count"].sum(), g.groupby(c_to)["count"].sum()], axis=1).fillna(0).sum(axis=1)
        keep = set(deg.nlargest(80).index)  # keep manageable
        g_c = g[g[c_from].isin(keep) & g[c_to].isin(keep)] if not deg.empty else g

        ents = sorted(set(g_c[c_from].astype(str)).union(set(g_c[c_to].astype(str))))
        if not ents:
            print("[info] No edges after capping — skipping Sankey.")
        else:
            idx = {name: i for i, name in enumerate(ents)}
            src = [idx[s] for s in g_c[c_from].astype(str)]
            tgt = [idx[t] for t in g_c[c_to].astype(str)]
            val = g_c["count"].tolist()
            link_label = g_c[c_rel].astype(str).tolist()

            fig = go.Figure(data=[go.Sankey(
                arrangement="snap",
                node=dict(label=ents, pad=20, thickness=16),
                link=dict(source=src, target=tgt, value=val, label=link_label))])
            fig.update_layout(title_text="Entity → Entity by Relation (Optional)",
                              width=1200, height=700)
            fig.show()
else:
    print("[info] Entity_Relationship_Edges.csv missing or empty — nothing to plot.")
